In [15]:
# instruciton datasetの生成

In [1]:
import os

#dataフォルダ内をリセット
os.system("mkdir data")
#os.system("rm -rf data/*")

mkdir: cannot create directory ‘data’: File exists


0

In [2]:
from datasets import load_dataset
import json

/home/hatakeyama/miniconda3/envs/llmeval/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 自動生成したQAの読み込み

In [3]:
dataset=load_dataset("hatakeyama-llm-team/AutoGeneratedJapaneseQA",split="train")

Generating train split: 165789 examples [00:00, 246422.38 examples/s]


In [4]:
print(len(dataset))

165789


In [5]:
def clean_autogen(text):
    if text is None:
        return ""
    text=text.strip()
    return text

In [6]:
#original template
question_template="以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。\n\n### 指示:\n"
answer_template="\n### 応答:\n"

#custom template
question_template="以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。<SEP>指示<SEP>"
answer_template="<SEP>応答<SEP>"

In [7]:
dataset[0]

{'question': '次の本を読んだことがある人物について、進化論や生物学に関する知識があるかどうか考察してください。『のとヒトの進化』',
 'database': 'hatakeyama-llm-team/WikiBookJa',
 'answer': ' まず、「の問題とヒトの進化」を読んだことがある人物については、それ自体が進化論や生物学に関する知識を持っている可能性が高いでしょう。この本は、ヒトの進化的起源についての深い分析と考察が含まれています。\n\n次に、進化論や生物学に関する知識を持っているかどうかを判断するためには、以下の点に注意してください。\n\n1. 進化論の基本概念: 人物が進化論の基礎的な概念や側面を理解しているかどうかを確認します。これには、遺伝子や突然変異、自然選択などが含まれます。\n2. 進化論の歴史: 人物が進化論の歴史的展開と重要な奮闘者について知っているかどうかを確認します。これには、ダーウィンやラマルクなどの名前が含まれます。\n3. 生物学的分類: 人物が生物学的分類法について理解しており、様々な生物種の進化的関係を把握できるかどうかを確認します。\n4. 現代進化論: 人物が最新の研究成果や分野内の議論について知っているかどうかを確認します。これには、Epigeneticsなどの新たな理解や進展が含まれます。\n5. 実際の例: 人物が進化論や生物学についての知識を実際の状況や現象に適用できるかどうかを確認します。これには、自然界で見られる様々な形式や行動が含まれます。\n\n上記の要素を踏まえて、「の問題とヒトの進化」を読んだことがある人物について、進化論や生物学に関する知識があるかどうかを調査します。',
 'score': 4.0}

In [26]:
#ベンチマークに用いるjmt benchと類似しすぎたデータは使わないようにする
import pandas as pd
#!pip install rapidfuzz
from rapidfuzz.process import cdist

jmt_bench_df=pd.read_csv("reference_data/jmtbench.csv")
bench_questions=jmt_bench_df["問い"].tolist()

def check_jmt_similarity(q,bench_questions):
    scores = cdist([q], bench_questions,workers=1)
    score=max(scores[0])
    return score

In [30]:
from tqdm import tqdm
score_threshold=4
sim_threshold=80
records=[]

for original_record in tqdm(iter(dataset)):
    q=clean_autogen(original_record["question"])
    a=clean_autogen(original_record["answer"])
    if original_record["score"] is None:
        continue
    if int(original_record["score"])<score_threshold:
        continue

    if check_jmt_similarity(q,bench_questions)>sim_threshold:
        print("too similar to jmt bench",q)
        continue

    text=f"{question_template}{q}{answer_template}{a}"
    if a!="":
        records.append(text)



len(records)

165789it [00:31, 5303.84it/s]


115600

In [31]:
#文法理解に関するデータセット
ds2=load_dataset("hatakeyama-llm-team/BumpoRikai",split="train")
ds2

Dataset({
    features: ['instruction', 'question', 'answer'],
    num_rows: 26448
})

In [32]:
bumpo_records=[]
for original_record in iter(ds2):
    q=clean_autogen(original_record["question"])
    a=clean_autogen(original_record["answer"])
    inst=clean_autogen(original_record["instruction"])
    
    text=f"{question_template}{inst}\n{q}{answer_template}{a}"

    bumpo_records.append(text)
print(text)

以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。<SEP>指示<SEP>次の質問への回答を生成。回答のみを記載してください。
問題　樹液からはチューインガムの原料が採取できる、中南米原産のアカテツ科の植物は何でしょう?<SEP>応答<SEP>サポジラ


In [33]:
import random

def write_jsonl(records,
    output_path="data/gousei.jsonl",
    n_eval=500,
    n_train=10**7,
    ):

    random.shuffle(records)

    lines=[json.dumps({"text":text},ensure_ascii=False) for text in records]
    with open (output_path,"w") as f:
        temp_lines=lines[:-n_eval]
        f.writelines(temp_lines[:n_train])

    lines=[json.dumps({"text":text},ensure_ascii=False) for text in records]
    with open (output_path+".eval","w") as f:
        f.writelines(lines[-n_eval:])


write_jsonl(records,"data/gousei.jsonl")
write_jsonl(bumpo_records,"data/bumpo.jsonl")
write_jsonl(records+bumpo_records,"data/gousei_bumpo.jsonl")